# Klasifikasi Kismis Menggunakan Deep Neural Network (DNN)


Proyek ini bertujuan untuk mengklasifikasikan jenis kismis (Besni dan Kecimen) berdasarkan fitur fitur morfologi menggunakan model Deep Neural Network (DNN) dan TensorFlow/Keras.

1. Persiapan Data

Pada tahap ini, kita memuat dataset, mengubah label teks menjadi angka, dan melakukan penskalaan fitur agar model saraf (neural network) dapat memproses data dengan lebih stabil.



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

# Memuat dataset
df = pd.read_csv('Raisin.csv')

# Memisahkan fitur (X) dan target (y)
X = df.drop('Class', axis=1)
y = df['Class']

# Mengubah label teks (Besni/Kecimen) menjadi 0/1
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Standarisasi data agar memiliki rata-rata 0 dan varians 1
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Pembagian data: 80% Training, 20% Testing
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

2. Membangun Arsitektur DNN

Kita menggunakan model sekuensial dengan teknik Batch Normalization (untuk mempercepat konvergensi) dan Dropout (untuk mencegah overfitting).

In [ ]:
model = models.Sequential([
    # Input Layer & Hidden Layer 1
    layers.Dense(89, activation='relu', input_shape=(X_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    # Hidden Layer 2
    layers.Dense(55, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    
    # Output Layer (Sigmoid cocok untuk klasifikasi biner)
    layers.Dense(1, activation='sigmoid')
])

# Kompilasi model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

3. Pelatihan Model (Training)

Menggunakan Early Stopping untuk menghentikan pelatihan jika tingkat kerugian (loss) pada data validasi tidak membaik selama 10 epoch berturut-turut.

In [ ]:
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    restore_best_weights=True
)

print("Memulai Pelatihan DNN...")
history = model.fit(
    X_train, y_train,
    epochs=36,
    batch_size=32,
    validation_split=0.2, # Mengambil 20% dari data train untuk validasi internal
    callbacks=[early_stopping],
    verbose=1
)

4. Evaluasi dan Visualisasi

Bagian ini menampilkan performa model dalam bentuk angka akurasi dan grafik visual.

In [ ]:
# Prediksi data test
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

# Print Laporan Akurasi
accuracy = accuracy_score(y_test, y_pred) * 100
print("\n" + "~" * 40)
print(f" >\\< Akurasi Model DNN: {accuracy:.2f}% >\\< ")
print("~" * 40 + "\n")
print(classification_report(y_test, y_pred, target_names=le.classes_))

# --- VISUALISASI ---

#  Confusion Matrix

plt.figure(figsize=(7, 6))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Confusion Matrix: Prediksi vs Aktual')
plt.xlabel('Prediksi')
plt.ylabel('Kenyataan')
plt.show()